In [1]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt

In [14]:
#### DEFINI OS VALORES DAS VARIÁVEIS INICIAIS
def set_up_configuration(normalize=(0,0.25),distribution_type='random'):
    rows, cols = 20, 20
    n_cells = rows * cols  

    # Parametros para função de mobilidade # Tipo de normalização padrão N(0,1)
    minN = normalize[0]
    maxN = normalize[1] 

    if maxN == 1 and minN == 0:
        raw_q = np.random.uniform(minN,maxN,n_cells)    
        raw_r = np.random.uniform(minN,1 - raw_q,n_cells)
        #raw_q = np.random.uniform(minN,maxN,n_cells)    
        #raw_r = np.random.uniform(minN,maxN,n_cells)
        #raw_i = np.random.uniform(minN,maxN,n_cells)
        #mask = (1 - (raw_r + raw_q + raw_i)) != 0
        #raw_r[mask] = raw_r[mask]/(raw_r[mask] + raw_q[mask] + raw_i[mask])
        #raw_q[mask] = raw_q[mask]/(raw_r[mask] + raw_q[mask] + raw_i[mask])
    else:
        raw_q = np.random.uniform(minN,maxN,n_cells)    
        raw_r = np.random.uniform(0, np.minimum(1, 1 - raw_q), n_cells)

    quitting_rate = raw_q
    forgetting_rate = raw_r

    rows_grid, cols_grid = np.mgrid[0:rows, 0:cols]  # Gera um grid
    row_indices = rows_grid.flatten()  # Transforma em array 1D [0, 0, ..., 19, 19]
    col_indices = cols_grid.flatten()  

    pop_status = np.zeros(n_cells, dtype=int)   ## inicia toda a população com status = 0 (S)
    disseminators = n_cells * 5 // 100  # 95% da populacao é S e 5% é I 

    if distribution_type == 'monopolistic':
        center_r, center_c = 17, 10 ##rows // 2, cols // 2
        count = 0
        for r_offset in range(-3, 2):
            for c_offset in range(-3, 2):
                if count < disseminators and -1 < (center_r + r_offset) < rows and -1 < (center_c + c_offset) < cols:
                    idx = (center_r + r_offset) * cols + (center_c + c_offset)
                    pop_status[idx] = 2
                    count += 1
    elif distribution_type == 'small-group':
        coords = [(3, 3), (3, 15), (10, 4), (13, 15), (18,0)]
        count = 0
        for gr, gc in coords:
            for r_offset in range(2):
                for c_offset in range(2):
                     if count < disseminators:
                        idx = (gr + r_offset) * cols + (gc + c_offset)
                        if idx < len(pop_status) and pop_status[idx] == 0:
                            pop_status[idx] = 2
                            count += 1
    else: # 'random'
        pop_status[:disseminators] = 2
        np.random.shuffle(pop_status)

    # Estados possíveis 0=S, 1=E, 2=I, 3=R, 4=Q 
    # Cria o dataframe com a população e seus Parametros individuais
    data = {
        'row': row_indices,
        'col' : col_indices,
        'status': pop_status,
        'learningCapability': np.abs(np.random.normal(loc=0.0, scale=1.0, size=n_cells)),
        'transferCapability': np.abs(np.random.normal(loc=0.0, scale=1.0, size=n_cells)),
        'forgettingRate': forgetting_rate, 
        'quittingRate': quitting_rate, 
        'timesContactor': np.zeros(n_cells).astype(int)
    }
    name_cols = ['row', 'col','status', 'learningCapability', 'transferCapability','forgettingRate', 'quittingRate', 'timesContactor']

    # Create DataFrame
    df = pd.DataFrame(data, columns=name_cols)
    return (df)

In [ ]:
cellularGrid1 = set_up_configuration()   ## padrão Random
cellularGrid2 = set_up_configuration(distribution_type = 'small-group')   
cellularGrid3 = set_up_configuration(distribution_type = 'monopolistic')

tipos=['Random','Small-Group','Monopolistic']
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
matriz1 = cellularGrid1.pivot(index='row', columns='col', values='status').values
matriz2 = cellularGrid2.pivot(index='row', columns='col', values='status').values
matriz3 = cellularGrid3.pivot(index='row', columns='col', values='status').values

axes[0].imshow(matriz1, cmap='binary')
axes[1].imshow(matriz2, cmap='binary')
axes[2].imshow(matriz3, cmap='binary')

# Configurar cada subplot
i = 0
for ax in axes:
    ax.grid(which='both', color='black', linestyle='-', linewidth=0.5, alpha=0.5)
    ax.set_title(f"{tipos[i]}")
    ax.set_xticks([])
    ax.set_yticks([])
    i+=1

plt.tight_layout()
plt.show()

In [3]:
def mobility(df, MD=20, IM=20, press=False):
    # Copy DataFrame to avoid modifying the original
    new_df = df.copy()

    max_row = df['row'].max()
    max_col = df['col'].max()
    rows = max_row + 1
    cols = max_col + 1

    if MD > max_row:
        MD = max_row

    columns_to_swap = [col for col in new_df.columns if col not in ['row', 'col']]
       
    # Calculate IMGrid: number of cells to swap = % da população afetada no movimento
    num_to_move = (len(df) * IM) // 100

    if MD == 0 or num_to_move == 0:
        if press:
            print("Mobilidade não aplicável (MD=0 ou IM=0). Trocas realizadas: 0")
        return new_df
    
    trocas = 0   
    # Perform swaps
    for _ in range(num_to_move):
        # Randomly select first cell (i, j)
        i = np.random.randint(0, max_row)
        j = np.random.randint(0, max_col)
        
        # Randomly select second cell (mdI, mdJ) within MDGrid 
        mdI = np.random.randint(-MD, MD+1)
        mdJ = np.random.randint(-MD, MD+1)
        while mdI == i and mdJ == j:  # Ensure different cells
            mdI = np.random.randint(-MD, MD+1)
            mdJ = np.random.randint(-MD, MD+1)

        mdI = (i + mdI) % rows # Fronteira toroidal
        mdJ = (j + mdJ) % cols # Fronteira toroidal

        # Find indices in DataFrame
        idx1 = new_df[(new_df['row'] == i) & (new_df['col'] == j)].index
        idx2 = new_df[(new_df['row'] == mdI) & (new_df['col'] == mdJ)].index
        
        if idx1.empty or idx2.empty:
            continue  # Skip if indices not found
        if idx1 == idx2:
            continue  # Skip if indices not found
        
        idx1, idx2 = idx1[0], idx2[0]    

        # Swap all attributes except row and col
        temp = new_df.loc[idx1, columns_to_swap].copy()
        new_df.loc[idx1, columns_to_swap] = new_df.loc[idx2, columns_to_swap]
        new_df.loc[idx2, columns_to_swap] = temp
        trocas += 1
    if press:
        print("Trocas realizadas: ",trocas)
        
    return new_df    

In [4]:
def neighborsF(df, point, radius=1, neighbors='Moore'):
    row, col = point
    neighbor_indices =[]
    rows = df['row'].max() + 1
    cols = df['col'].max() + 1   
    
    if neighbors not in ['Moore', 'VonNeumann']:
        raise ValueError("Escolha Moore ou Von Neumann")
    
    for dx in range(- radius, radius + 1):
        for dy in range(- radius, radius + 1):
            if dx == 0 and dy == 0:
                continue  # ignora a própria célula

            nx = (row + dx) % rows
            ny = (col + dy) % cols
            
            if neighbors == "VonNeumann":
                dist = abs(dx) + abs(dy)  # distância de Manhattan para validar o vizinho
                if dist <= radius:
                    dist = math.sqrt(dx**2 + dy**2) # distância Euclidiana
                    neighbor_indices.append([df[(df['row'] == nx) & (df['col'] == ny)].index,dist])

            elif neighbors == "Moore":  # Moore inclui todas as células no quadrado
                dist = math.sqrt(dx**2 + dy**2)  # distância Euclidiana
                neighbor_indices.append([df[(df['row'] == nx) & (df['col'] == ny)].index,dist])
        
    return neighbor_indices

In [5]:
def acquisition_rate(df, idx_current, radius, neighbors):
    # LearningCapability do ponto atual 
    row = df.loc[idx_current, 'row']
    col = df.loc[idx_current, 'col']
    point = (row,col)
    lc_current = df.loc[idx_current, 'learningCapability']

    # Obter vizinhos com suas distâncias
    neighbor_data = neighborsF(df, point, radius=radius, neighbors=neighbors)

    max_tax = 0
    for indices, dist in neighbor_data:
        if dist == 0:
            continue
        idx = indices[0]  # índice do vizinho no DataFrame

        status_vizinho = df.loc[idx, 'status']
        if status_vizinho == 2:  # status = I
            tc_vizinho = df.loc[idx, 'transferCapability']   
            # knowledge acquisition rate
            #tax = (1 / dist) * math.sqrt(lc_current * tc_vizinho) ## tá fazendo com que o nro de disseminadores atingam um topo rápido
            tax = (1 / dist) * (lc_current * tc_vizinho)

            if tax > max_tax:
                max_tax = tax

    return max_tax

In [6]:
def transition_F(df, radius, neighbors):
    new_df = df.copy() # cria uma cópia dos dados para promover as alterações no passo seguinte
    indices = df.index

    for idx in indices:
        status, D = df.loc[idx, ['status', 'timesContactor']]
        #if D==0: ### evitar divisão por 0
        #    D=1
        
        ## Transição de S para E  
        if status == 0:
            taxa = acquisition_rate(df, idx, radius, neighbors)
            # Se encontrou pelo menos um vizinho com status 2, sortear um numero entre 0 e 1 e se for maior troca
            if taxa > np.random.random():
                new_df.at[idx, 'status'] = 1  # transita de S (0) para E (1)
                new_df.at[idx, 'timesContactor'] += 1

        ## Transição de E para I, R ou Q
        elif status == 1:

            txR_base = df.loc[idx, 'forgettingRate']
            txQ_base = df.loc[idx, 'quittingRate']

            # Ajuste com reforço de aprendizado, conforme o artigo 
            Rl = txR_base ** (D + 1)
            Ql = txQ_base ** (D + 1)
    
            # A taxa de absorção (Il) deve aumentar. O artigo define I' = 1 - R' - Q' 
            # Isso significa que o que não é perdido para R ou Q, se torna I.
            Il = 1 - Rl - Ql

            # Normalização em caso de erro numérico
            if Il < 0:
                Il = 0
                total = Rl + Ql
                Rl /= total
                Ql /= total

            # Sorteio do destino
            r = np.random.rand()
            if r < Ql:
                new_df.at[idx, 'status'] = 4  # Q - Quitter
            elif r < Ql + Rl:
                lc_current = df.loc[idx, 'learningCapability']
                new_df.at[idx, 'status'] = 3  # R - Forgetter
                ## incrementa a capacidade de aprendizado 
                new_df.at[idx, 'learningCapability'] = lc_current ** (1.0 / (D+1.0)) if D >= 0 and lc_current > 0 else lc_current
            else:
                new_df.at[idx, 'status'] = 2  # I - Disseminator
        ## transição de R para E
        elif status == 3:         
            taxa = acquisition_rate(df, idx, radius, neighbors)
            if taxa > np.random.random():
                new_df.at[idx, 'status'] = 1  # E - Contactor
    return new_df

In [7]:
def simular_difusao(df, MD=20, IM=50, radius=1, neighbors='Moore', press=False):
    max_iter = 50
    history_stats = [] # resetar histórico

    for passo in range(1, max_iter + 1):
        status_antes = df['status'].value_counts().to_dict()

        df = transition_F(df, radius, neighbors)
        df = mobility(df,MD,IM)

        status_depois = df['status'].value_counts().to_dict()

        # Salva os dados para gerar estatísticas
        history_stats.append({
            'ciclo': passo,
            **{f'status_{s}': status_depois.get(s, 0) for s in range(5)}
        })

        if press and (passo % 10 == 0):
            print(f'\n🌀 Ciclo {passo}')
            print(f'Estatísticas:')
            for s in range(5):
                antes = status_antes.get(s, 0)
                depois = status_depois.get(s, 0)
                print(f'  Estado {s}: {antes} → {depois}')

        if status_antes == status_depois and press:
            break
    if press:
        print(f'⛔️ Execução encerrada no ciclo de tempo {passo}.')
    return pd.DataFrame(history_stats)

In [8]:
def plot_results(results_dict, title_prefix, filename, colors=None, linestyles=None, n_cells=400):
    """
    Função genérica para plotar os resultados de qualquer experimento.
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    for name, df_history in results_dict.items():
        df_history['r_t'] = df_history['status_2'] / n_cells
        df_history['v_t'] = df_history['status_2'].diff().fillna(0)
        ax1.plot(df_history['ciclo'], df_history['r_t'], label=name,
                 color=colors.get(name) if colors else None,
                 linestyle=linestyles.get(name) if linestyles else '-', marker='o', markersize=3)
        ax2.plot(df_history['ciclo'], df_history['v_t'], label=name,
                 color=colors.get(name) if colors else None,
                 linestyle=linestyles.get(name, '--') if linestyles else '--', marker='x', markersize=4)
    ax1.set_title('(a) Proporção de Disseminadores vs. Tempo')
    ax1.set_xlabel('Ciclos de Tempo')
    ax1.set_ylabel('Proporção de Disseminadores ($r_t$)')
    ax1.grid(True, linestyle='--', alpha=0.6)
    ax1.legend()
    ax1.set_ylim(0, 1.05)
    ax1.set_xlim(0, 50)
    ax2.set_title('(b) Velocidade de Difusão vs. Tempo')
    ax2.set_xlabel('Ciclos de Tempo')
    ax2.set_ylabel('Novos Disseminadores ($v_t$)')
    ax2.grid(True, linestyle='--', alpha=0.6)
    ax2.legend()
    ax2.set_xlim(0, 50)
    fig.suptitle(title_prefix, fontsize=16)
    fig.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.savefig(filename, dpi=300)
    print(f"Gráficos salvos com sucesso no arquivo '{filename}'")
    plt.close(fig)

In [9]:
# --- FUNÇÕES DE EXECUÇÃO DOS EXPERIMENTOS ---
def run_experiment_figure_4(MD=0,IM=0,radius=1,neighbors='Moore',rodadas=50):
    """Executa as simulações para replicar a Figura 4."""
    print("\n--- Iniciando simulações para a Figura 4: Impacto da Distribuição Inicial ---")
    NUM_RODADAS = rodadas
    scenarios = ['random', 'small-group', 'monopolistic']
    results = {}
    for scenario in scenarios:
        print(f"  Executando cenário: {scenario.capitalize()} - ({NUM_RODADAS} rodadas)...")
        rodadas_history = []
        for i in range(NUM_RODADAS):
            np.random.seed(i) # Semente diferente para cada rodada
            initial_grid = set_up_configuration(distribution_type=scenario)  
            history = simular_difusao(initial_grid, MD=MD, IM=IM, radius=radius
                                     ,neighbors=neighbors, press=False)
            rodadas_history.append(history)            
        # Calcula a média dos resultados
        avg_history = pd.concat(rodadas_history).groupby('ciclo').mean().reset_index()
        results[scenario.capitalize()] = avg_history
    plot_results(results,
                 'Impacto da Distribuição Inicial dos Disseminadores',
                 'figura_4_simulacao.png',
                 colors={'Random': 'green', 'Small-group': 'orange', 'Monopolistic': 'blue'},
                 linestyles={'Random': '-', 'Small-group': '--', 'Monopolistic': ':'})

def run_experiment_figure_5(MD=5,IM=0,rodadas=50):
    """Executa as simulações para replicar a Figura 5."""
    print("\n--- Iniciando simulações para a Figura 5: Impacto da Vizinhança ---")
    NUM_RODADAS = rodadas
    scenarios = {
        '1x1 Von Neumann': {'radius': 1, 'type': 'VonNeumann'},
        '1x1 Moore':       {'radius': 1, 'type': 'Moore'},
        '2x2 Von Neumann': {'radius': 2, 'type': 'VonNeumann'},
        '2x2 Moore':       {'radius': 2, 'type': 'Moore'}
    }
    results = {}
    for name, params in scenarios.items():
        print(f"  Executando cenário: {name} - ({NUM_RODADAS} rodadas)...")
        rodadas_history = []
        for i in range(NUM_RODADAS):
            np.random.seed(i) # Semente diferente para cada rodada
            initial_grid = set_up_configuration()  
            history = simular_difusao(initial_grid.copy(), MD=MD, IM=IM,radius=params['radius'],
                                      neighbors=params['type'], press=False)
            rodadas_history.append(history)
        # Calcula a média dos resultados
        avg_history = pd.concat(rodadas_history).groupby('ciclo').mean().reset_index()
        results[name] = avg_history
    plot_results(results,
                 'Impacto da Acessibilidade ao Conhecimento (Vizinhança)',
                 'figura_5_simulacao.png')

def run_experiment_figure_6(MD=5,radius=1,neighbors='Moore',rodadas=50):
    """Executa as simulações para replicar a Figura 6 (Impacto da Proporção de Indivíduos Móveis)."""
    print("\n--- Iniciando simulações para a Figura 6: Impacto da Proporção de Móveis (IM) ---")
    NUM_RODADAS = rodadas
    scenarios = [0, 20, 50, 100]
    results = {}

    for im_percent in scenarios:
        name = f'IM = {im_percent}%'
        print(f"  Executando cenário: {name} - ({NUM_RODADAS} rodadas)...")
        rodadas_history = []
        for i in range(NUM_RODADAS):
            np.random.seed(i) # Semente diferente para cada rodada
            initial_grid = set_up_configuration()  
            history = simular_difusao(initial_grid.copy(), MD=MD, IM=im_percent, radius=radius
                                     ,neighbors=neighbors, press=False)
            rodadas_history.append(history)
        # Calcula a média dos resultados
        avg_history = pd.concat(rodadas_history).groupby('ciclo').mean().reset_index()
        results[name] = avg_history
    plot_results(results,
                 'Impacto da Proporção de Indivíduos Móveis (IM)',
                 'figura_6_simulacao.png')

def run_experiment_figure_7(IM=20,radius=1,neighbors='Moore',rodadas=50):
    """Executa as simulações para replicar a Figura 7 (Impacto da Distância Máxima de Movimento)."""
    print("\n--- Iniciando simulações para a Figura 7: Impacto da Distância de Movimento (MD) ---")
    NUM_RODADAS = rodadas
    scenarios = [0, 5, 10, 20]
    results = {}
    for md_dist in scenarios:
        name = f'MD = {md_dist}'
        print(f"  Executando cenário: {name} - ({NUM_RODADAS} rodadas)...")
        rodadas_history = []
        for i in range(NUM_RODADAS):
            np.random.seed(i) # Semente diferente para cada rodada
            initial_grid = set_up_configuration()  
            history = simular_difusao(initial_grid.copy(), 
                                  MD=md_dist, IM=IM,
                                  radius=radius, neighbors=neighbors, press=False)
            rodadas_history.append(history)
        # Calcula a média dos resultados
        avg_history = pd.concat(rodadas_history).groupby('ciclo').mean().reset_index()
        results[name] = avg_history
    plot_results(results,
                 'Impacto da Distância Máxima de Movimento (MD)',
                 'figura_7_simulacao.png')

def run_experiment_figure_8(MD=20,IM=100,radius=1,neighbors='Moore',rodadas=50):
    """Executa as simulações para replicar a Figura 8 (Impacto da Taxa de Desistência)."""
    print("\n--- Iniciando simulações para a Figura 8: Impacto da Taxa de Desistência ---")
    NUM_RODADAS = rodadas
    # Interpretando a notação N(a,b) do artigo como uma distribuição Uniforme(a,b)
    # cenários do artigo seção 4.4
    scenarios = {
        'Taxa [0.00-0.25]': {'min': 0.00, 'max': 0.25},
        'Taxa [0.25-0.50]': {'min': 0.25, 'max': 0.50},
        'Taxa [0.50-0.75]': {'min': 0.50, 'max': 0.75},
        'Taxa [0.75-1.00]': {'min': 0.75, 'max': 1.00} ## no artigo menciona 0,75 a 0,10 (mas acho que é 0,75 a 1)
    }
    results = {}
    for name, params in scenarios.items():
        print(f"  Executando cenário: {name} - ({NUM_RODADAS} rodadas)...")
        rodadas_history = []
        for i in range(NUM_RODADAS):
            np.random.seed(i) # Semente diferente para cada rodada
            initial_grid = set_up_configuration(normalize=(params['min'],params['max']))
            history = simular_difusao(initial_grid.copy(), MD=MD, IM=IM,
                                      radius=radius, neighbors=neighbors, press=False)
            rodadas_history.append(history)
        # Calcula a média dos resultados
        avg_history = pd.concat(rodadas_history).groupby('ciclo').mean().reset_index()
        results[name] = avg_history
    plot_results(results,
                 'Impacto da Taxa de Desistência do Conhecimento',
                 'figura_8_simulacao.png')

In [17]:
#run_experiment_figure_4(rodadas=2)
#run_experiment_figure_5(rodadas=2)
run_experiment_figure_6(rodadas=2)
run_experiment_figure_7(rodadas=2)
#run_experiment_figure_8(rodadas=50)

print("\nTodos os experimentos foram concluídos com sucesso.")


--- Iniciando simulações para a Figura 6: Impacto da Proporção de Móveis (IM) ---
  Executando cenário: IM = 0% - (2 rodadas)...
  Executando cenário: IM = 20% - (2 rodadas)...
  Executando cenário: IM = 50% - (2 rodadas)...
  Executando cenário: IM = 100% - (2 rodadas)...
Gráficos salvos com sucesso no arquivo 'figura_6_simulacao.png'

--- Iniciando simulações para a Figura 7: Impacto da Distância de Movimento (MD) ---
  Executando cenário: MD = 0 - (2 rodadas)...
  Executando cenário: MD = 5 - (2 rodadas)...
  Executando cenário: MD = 10 - (2 rodadas)...
  Executando cenário: MD = 20 - (2 rodadas)...
Gráficos salvos com sucesso no arquivo 'figura_7_simulacao.png'

Todos os experimentos foram concluídos com sucesso.
